# HUMAN PPI 


### PIPELINE OVERVIEW
+ graph with genes as nodes and edges
+ generate a matrix from nodes based on specific parameters (features, random walk, shortest path..)
+ multidimensional matrices > embedding with tsne into 2D or 3D 
----------
+ Visualization typologies:
+ 2D PORTRAIT
+ 3D PORTRAIT
+ 3D LANDSCAPE
+ 3D SPHERE
----------

In [ ]:
from multidimvis_main import *

In [ ]:
organism = 'Human'

In [ ]:
G = nx.read_edgelist('input/ppi_elist.txt',data=False)
# d_ent_sym, d_sym_ent = genent2sym()

d_gene_do = pickle.load( open( "input/d_gene_do.pkl", "rb" ) )
d_do_genes = pickle.load( open( "input/d_do_genes.pkl", "rb" ) )
d_do_names = pickle.load( open( "input/DO_names.pkl", "rb" ) )
d_names_do = {y:x for x,y in d_do_names.items()}

In [ ]:
print('Number of nodes i.e. genes: %s' %len(list(G.nodes())))
print('Number of edges: %s' %len(list(G.edges())))
print('Network density: %.1f%%' %(200.*len(list(G.edges()))/(len(list(G.nodes()))*len(list(G.nodes()))-1)))

_____
# FEATURES
_____

### + STRUCTURAL : Graph based

In [ ]:
DM_adj = pd.read_csv('output_csv/Adjacency_Dataframe_Human.csv', index_col=0)

In [ ]:
DM_spl = pd.read_csv('output_csv/SPL_Dataframe_Human.csv', index_col=0)

In [ ]:
DM_m = pd.read_csv('output_csv/RWR_Dataframe_Human.csv', index_col=0)

### + STRUCTURAL : Centrality measures

In [ ]:
# precalculated import
df_centralities = pd.read_csv('output_csv/Features_centralities_Dataframe_'+organism+'.csv', index_col=0)

d_deghubs = dict(zip(G.nodes(),df_centralities['degs']))
d_clos = dict(zip(G.nodes(), df_centralities['clos']))
d_betw = dict(zip(G.nodes(), df_centralities['betw']))
d_eigen = dict(zip(G.nodes(), df_centralities['eigen']))

d_centralities = dict(zip(list(G.nodes),zip(d_deghubs.values(),d_clos.values(),d_betw.values(),d_eigen.values())))
l_features = []
for i in d_centralities.items():
    k=list(i)
    l_features.append(k)
    
DM_centralities = pd.DataFrame(distance.squareform(distance.pdist(df_centralities, 'euclidean')))

# DM_centralities.to_csv(r'output_csv/DistanceMatrix_centralities_Dataframe_'+organism+'.csv', index = True)
# DM_centralities = pd.read_csv('output_csv/DistanceMatrix_centralities_Dataframe_'+organism+'.csv', index_col=0)

### + FUNCTIONAL : Molecular Function

In [ ]:
#df_MF = pd.read_csv('output_csv/Features_GO_MolFunc_Dataframe_Human.csv', index_col=0)
#MF_array = df_MF.to_numpy()
#DM_MF = pd.DataFrame(distance.squareform(distance.pdist(MF_array, 'euclidean')))
#DM_MF.to_csv(r'output_csv/DistanceMatrix_goMF_Dataframe_'+organism+'.csv', index = True)

DM_MF = pd.read_csv('output_csv/DistanceMatrix_goMF_Dataframe_'+organism+'.csv', index_col=0)

### + FUNCTIONAL : Cellular Components

In [ ]:
#df_CC = pd.read_csv('output_csv/Features_GO_CellComp_Dataframe_Human.csv', index_col=0)
#CC_array = df_CC.to_numpy()
#DM_CC = pd.DataFrame(distance.squareform(distance.pdist(CC_array, 'euclidean')))
#DM_CC.to_csv(r'output_csv/DistanceMatrix_goCC_Dataframe_'+organism+'.csv', index = True)

DM_CC = pd.read_csv('output_csv/DistanceMatrix_goCC_Dataframe_'+organism+'.csv', index_col=0)

### + FUNCTIONAL : Biological Processes

In [ ]:
#df_BP = pd.read_csv('output_csv/Features_GO_BiolProc_Dataframe_Human.csv', index_col=0)
#BP_array = df_BP.to_numpy()
#DM_BP = pd.DataFrame(distance.squareform(distance.pdist(BP_array, 'euclidean')))
#DM_BP.to_csv(r'output_csv/DistanceMatrix_goBP_Dataframe_'+organism+'.csv', index = True)

DM_BP = pd.read_csv('output_csv/DistanceMatrix_goBP_Dataframe_'+organism+'.csv', index_col=0)

### + FUNCTIONAL : Disease Annotation

In [ ]:
#df_DIS = pd.read_csv('output_csv/Features_Disease_Dataframe_'+organism+'.csv', index_col=0)
#DIS_array = df_DIS.to_numpy()
#DM_disease = pd.DataFrame(distance.squareform(distance.pdist(DIS_array, 'euclidean')))
#DM_disease.to_csv(r'output_csv/DistanceMatrix_Disease_Dataframe_'+organism+'.csv', index = True)

DM_disease = pd.read_csv('output_csv/DistanceMatrix_Disease_Dataframe_'+organism+'.csv', index_col=0)
genes_dis = list(DM_disease.index)

_____

### SELECT DISTANCE MATRIX

In [ ]:
# G R A P H - B A S E D 
#feature = 'aij'
#DM = DM_adj

#feature = 'spl'
#DM = DM_spl

feature = 'rwr'
DM = DM_m

# C E N T R A L I T I E S
#feature = 'centralities'
#DM = DM_centralities


# D I S E A S E S 
#feature = 'disease'
#DM = DM_disease 


# G O T E R M S (MF; BP; CC)
#feature = 'MF'
#DM = DM_MF

#feature = 'BP'
#DM = DM_BP

#feature = 'CC'
#DM = DM_CC

____________
# VISUALIZATION SETTINGS
____________

### NODE SIZE

In [ ]:
def draw_node_degree_3D(G, l_genes, scalef):
    x = 20
    ring_frac = (x-1.)/x

    deg = dict(G.degree())
    
    l_size = {}
    for i in l_genes:
        for k,v in deg.items():
            if i == k:
                R = scalef * (1+v**1.5)
                r = ring_frac * R
                l_size[i] = r
    
    return l_size

In [ ]:
node_factor = 2
size = draw_node_size(G, d_deghubs, node_factor) # 2D

node_factor3d = 2
size3d = list(draw_node_degree_3D(G, list(G.nodes()), node_factor3d).values())

In [ ]:
#edge_color = 'grey'
edge_width = 0.1
opacity_nodes = 1
opacity_edges = 0.9

### COLOUR PARAMETERS

+ DISEASE COLOURED

In [ ]:
disease = 'lymphoma'
# DOID:707 B-Cell Lymphoma (71 genes)
# DOID:9252 amino acids metabolic disorder (130 genes)
# DOID:14320 generalized anxiety disorder (112 genes)

disease_col = 'red'
disease_genes = get_disease_genes(G, d_names_do, d_do_genes, disease)

colours = color_diseasecategory(G, d_names_do, d_do_genes, disease, disease_col)
node_edge_col = colours

In [64]:
def color_disease_outgoingedges(G, l_majorcolor_nodes, color):
    
    d_col_major = {}
    for idx,n in enumerate(l_majorcolor_nodes):
            d_col_major[n] = color

    # Node outgoing edges
    edge_lst = []
    for edge in G.edges():
        for e in edge:
            for node in d_col_major.keys():
                if e == node:
                    edge_lst.append(edge)

    # Color edges based on major nodes
    d_col_edges = {}
    for e in edge_lst:
        for node,col in d_col_major.items():
            if e[0] == node:
                d_col_edges[e]=col
            elif e[1] == node:
                d_col_edges[e]=col

    d_grey_edges = {}
    for edge in G.edges():
        if edge not in d_col_edges.keys(): 
            d_grey_edges[edge] = None

    d_edges_all = {**d_col_edges, **d_grey_edges}

    # Sort according to G.edges()
    d_edges_all_sorted = {key:d_edges_all[key] for key in G.edges()}

    edge_color = list(d_edges_all_sorted.values())
    
    return edge_color

In [65]:
edge_color = color_disease_outgoingedges(G, disease_genes, disease_col)

+ HUBS COLOURED

In [ ]:
cutoff = 3 # number of considered hub nodes 
d_hub_nodes = identify_hubs(d_deghubs, d_clos, d_betw, cutoff)

colours = color_majornodes(G, d_hub_nodes)
node_edge_col = colours

edge_color = color_majornodes_outgoingedges(G, d_hub_nodes)

+ CLUSTERS COLOURED

+ CLOSENESS CENTRALITY COLOR 

In [ ]:
# Define colour parameter
d_to_be_coloured = d_clos # dict sorted by dict.values (that way the biggest value matches darkest colour of palette)


# Colouring
colour_groups = set(d_to_be_coloured.values())
colour_count = len(colour_groups)
palette = sns.color_palette('Reds', colour_count)

d_colourgroups = {}
for n in colour_groups:
    d_colourgroups[n] = [k for k in d_to_be_coloured.keys() if d_to_be_coloured[k] == n]
    
d_colourgroups_sorted = {key:d_colourgroups[key] for key in sorted(d_colourgroups.keys())}

d_val_col = {}
for idx,val in enumerate(d_colourgroups_sorted):
    for ix,v in enumerate(palette):
        if idx == ix:
            d_val_col[val] = v
d_node_colour = {}
for y in d_to_be_coloured.items(): # y[0] = node id, y[1] = val
    for x in d_val_col.items(): # x[0] = val, x[1] = (col,col,col)
        if x[0] == y[1]:
            d_node_colour[y[0]]=x[1]
            
# SORT dict based on G.nodes
d_node_colour_sorted = dict([(key, d_node_colour[key]) for key in G.nodes()])

l_col_clos = list(d_node_colour_sorted.values())

colours = l_col_clos 

____________
# 2D LAYOUTS
____________

## PORTRAITS

### tSNE EMBEDDING 2D

In [ ]:
prplxty = 50 # range: 5-50 / default = 30
density = 1.2 # default 12.
l_rate = 200 # default 200.
steps = 250 # min 250
metric = 'euclidean'

In [ ]:
%%time 
tsne = embed_tsne_2D(DM, prplxty, density, l_rate, steps, metric)

In [ ]:
%%time 
posG_disease = get_posG_2D(genes_dis, tsne)

### work in progress: Disease annotation layout - how to plot rest nodes 

In [ ]:
# might be replaced by mapping rest nodes to some geometry 

coords_disease = list(posG_disease.values())

x_max = max(coords_disease)[0]
x_min = min(coords_disease)[0]
y_max = max(coords_disease)[1]
y_min = min(coords_disease)[1]

rand_coords = [(np.random.uniform(x_min,x_max), np.random.uniform(y_min,y_max)) for i in range(len(genes_rest))]

In [ ]:
import numpy.linalg as la

t = np.linspace(0,2*np.pi,len(genes_rest))
xc = 1*np.random.rand() #x-coordinate of center of circle 
yc = 1*np.random.rand() #y-coordinate of center of circle
r = 6*np.random.rand() + 0.5 #radius of circle

x = r*np.cos(t) + np.random.normal(scale = 1.0/7, size = len(t)) + xc  
y = r*np.sin(t) + np.random.normal(scale = 1.0/7, size = len(t)) + yc

In [ ]:
genes_rest = []
for g in G.nodes():
    if int(g) not in posG_disease.keys():
        genes_rest.append(str(g))
        
posG_rest = {}
for g in genes_rest:
    for c in rand_coords:
        posG_rest[str(g)] = c

posG_all = {**posG_disease, **posG_rest}
posG_tsne = {key:posG_all[key] for key in G.nodes()}

### 2D tSNE PORTRAITS

In [ ]:
plt.figure(figsize=(25,25))
plt.title(feature+' | t-SNE | Metric: '+metric, size= 16)

nx.draw_networkx_nodes(G, posG_tsne, edgecolors = node_edge_col, linewidths = 0.5, node_color=colours, node_size=size, alpha = opacity_nodes)
nx.draw_networkx_edges(G, pos = posG_tsne, width = edge_width, edge_color = edge_color, alpha = opacity_edges)
#nx.draw_networkx_labels(G, pos = posG, font_size = fontsize_labels, font_color = 'black')
plt.box(False)

plt.savefig('output_plots/2Dtsne_'+feature+'_'+metric+'_'+organism+'.png')

plt.show()

### UMAP EMBEDDING 2D 
+https://umap-learn.readthedocs.io/en/latest/embedding_space.html

In [ ]:
n_neighbors = 10 # balance between local and global structure in the data
spread = 2.
min_dist = 0.5 # defines how dense points are stacked together 
n_components = 2 # for 2D

U = umap.UMAP(
        n_neighbors = n_neighbors,
        spread = spread,
        min_dist = min_dist,
        n_components = n_components,
        metric = metric)

In [ ]:
%%time 
umap = U.fit_transform(DM)
posG_umap = get_posG_2D(list(G.nodes(),umap)

### 2D UMAP PORTRAITS

In [ ]:
plt.figure(figsize=(25,25))
plt.title('PW dist EUCLIDEAN | UMAP | Metric: '+metric, size=16)

nx.draw_networkx_nodes(G, posG_umap_eucl, edgecolors = node_edge_col, linewidths = 0.5, node_color=colours, node_size=size, alpha = 1)
nx.draw_networkx_edges(G, pos = posG_umap_eucl, width = edge_width, edge_color = edge_color, alpha = opacity_edges)
#nx.draw_networkx_labels(G, pos = posG_umap_spl, font_size = fontsize_labels, font_color = 'black')
plt.box(False)

plt.savefig('output_plots/2Dumap_eucl_'+metric+'_'+organism+'.png')

plt.show()

## LANDSCAPES

In [ ]:
#dict_z = {key:d_essentiality_scores[key] for key in G.nodes()}
dict_z = {key:d_clos[key] for key in G.nodes()}

z_list = list(dict_z.values())

### tSNE

In [ ]:
%%time

tsne_x, tsne_y, tsne_z = get_coords_landscape(G, posG_tsne)
tsne_trace_z = get_trace_nodes_landscape(tsne_x, tsne_y, z_list, colours, size3d)
tsne_data = [tsne_trace_z] #,adj_edges]

In [ ]:
print('Perplexity:', prplxty)
print('Early Exaggeration:', density)
print('Learning rate:', l_rate)
print('Iterations:', steps)

fig = pgo.Figure()
for i in tsne_data:
    fig.add_trace(i)

fig.update_layout(template='plotly_dark', height = 1000, width = 2200)
#fig.update_layout(template=None, height = 1000, width = 2200)

py.iplot(fig)

plotly.offline.plot(fig_1, filename = 'output_plots/Landscapes_tsne_'+feature+'_'+metric+'_'+organism+'.html', auto_open=False)

### UMAP

In [ ]:
%%time

umap_x, umap_y, umap_z = get_coords_landscape(G, posG_umap_adj)
umap_trace_z = get_trace_nodes_landscape(umap_x, umap_y, z_list, colours, size3d)
umap_data = [umap_trace_z]

In [ ]:
print('# Neighbors: ', n_neighbors)
print('Spread: ', spread)
print('Min. Distance: ', min_dist)
print('Metric: ', metric)


fig = pgo.Figure()
for i in umap_data:
    fig.add_trace(i)

fig.update_layout(template='plotly_dark', height = 1000, width = 2200)
#fig.update_layout(template=None, height = 1000, width = 2200)

py.iplot(fig)

plotly.offline.plot(fig_1, filename = 'output_plots/Landscapes_umap_'+feature+'_'+metric+'_'+organism+'.html', auto_open=False)

### LANDSCAPE | EXPORT Coordinates x,y,z,rgba

In [ ]:
colours_r = []
colours_g = []
colours_b = []
colours_a = []
for i in colours:
    colours_r.append(i[0])
    colours_g.append(i[1])
    colours_b.append(i[2])
    colours_a.append(0.9)

In [ ]:
df_xyz_landscape_tsne = pd.DataFrame(posG_tsne, index=["x","y"]).T
df_xyz_landscape_tsne["z"] = z_list

df_xyz_landscape_tsne['r'] = colours_r
df_xyz_landscape_tsne['g'] = colours_g
df_xyz_landscape_tsne['b'] = colours_b
df_xyz_landscape_tsne['a'] = colours_a

df_xyz_landscape_tsne.to_csv(r'output_csv/landscape_tsne_XYZ_RGBA_'+feature+_'+organism+'.csv', index = True)

In [ ]:
df_xyz_landscape_umap = pd.DataFrame(posG_umap, index=["x","y"]).T
df_xyz_landscape_umap["z"] = z_list

df_xyz_landscape_umap['r'] = colours_r
df_xyz_landscape_umap['g'] = colours_g
df_xyz_landscape_umap['b'] = colours_b
df_xyz_landscape_umap['a'] = colours_a

df_xyz_landscape_umap.to_csv(r'output_csv/landscape_umap_XYZ_RGBA_'+feature+_'+organism+'.csv', index = True)

____________
# 3D LAYOUTS
____________


## PORTRAITS

In [67]:
prplxty3d = 50 # range: 5-50 / default = 30.#
density3d = 1.2 # default 12.
l_rate3d = 200 # default 200.
steps3d = 250 #min 250

metric = 'euclidean'
#metric = 'correlation'
#metric = 'cosine'

#### tSNE

In [ ]:
'''%%time
tsne = embed_tsne_3D(DM, prplxty3d, density3d, l_rate3d, steps3d, metric)
posG3d = get_posG_3D(G, tsne)

# EXPORT as dataframe 
df_3Dportrait_tsne = pd.DataFrame(posG3d).T
df_3Dportrait_tsne.columns = ['X','Y','Z']
df_3Dportrait_tsne.to_csv(r'output_csv/3DPortrait_tsne_'+feature+'_Human_lrate_'+str(l_rate3d)+'steps_'+str(steps3d)+'_'+metric+'.csv', index = True)'''

In [68]:
# IMPORT embedded MARKOV Matrix 

df_tsne_posG3d = pd.read_csv('output_csv/3DPortrait_tsne_'+feature+'_Human_'+'lrate_'+str(l_rate3d)+'steps_'+str(steps3d)+'_'+metric+'.csv', index_col=0)
X_tsne = list(df_tsne_posG3d['X'])
Y_tsne = list(df_tsne_posG3d['Y'])
Z_tsne = list(df_tsne_posG3d['Z'])

tsne_posG3d = dict(zip(list(G.nodes()),zip(X_tsne ,Y_tsne ,Z_tsne)))

In [70]:
def get_trace_edges_(G, posG, color_list):
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in G.edges():
        for i in color_list:
            if i != None: 
                x0, y0, z0 = posG[edge[0]]
                x1, y1, z1 = posG[edge[1]]
                edge_x.append(x0)
                edge_x.append(x1)
                edge_x.append(None)
                edge_y.append(y0)
                edge_y.append(y1)
                edge_y.append(None)
                edge_z.append(z0)
                edge_z.append(z1)
                edge_z.append(None)
    
    trace_edges = pgo.Scatter3d(
                        x = edge_x, 
                        y = edge_y, 
                        z = edge_z,
                        mode = 'lines', hoverinfo='none',
                        line = dict(width = 0.5, color = color_list),
                        opacity = 0.3
                )
    
    return trace_edges

In [ ]:
%%time

tsne_edges = get_trace_edges_(G, tsne_posG3d, edge_color) 
tsne_nodes = get_trace_nodes(G, tsne_posG3d, l_features, colours, size3d)

#tsne_data = [tsne_edges, tsne_nodes]
tsne_data = [tsne_nodes]

In [ ]:
tsne_data = [tsne_edges, tsne_nodes]

In [ ]:
annotation_text = 'diseases'

In [ ]:
print('Perplexity:', prplxty3d)
print('Early Exaggeration:', density3d)
print('Learning rate:', l_rate3d)
print('Iterations:', steps3d)

fig = pgo.Figure()
for i in tsne_data:
    fig.add_trace(i)
fig.update_layout(template='plotly_dark', showlegend=False, width=2000, height=2000,
                  scene=dict(
                    aspectratio=dict(
                        x=1,
                        y=1,
                        z=1
                    ),
                    camera=dict(
                        center=dict(
                            x=0,
                            y=0,
                            z=0
                        ),
                        eye=dict(
                            x=1.96903462608,
                            y=-1.09022831971,
                            z=0.405345349304
                        ),
                        up=dict(
                            x=0,
                            y=0,
                            z=1
                        )
                    ),
                    dragmode="turntable",
                    annotations=[dict(
                        showarrow=True,
                        x=0,
                        y=-10,
                        z=10,
                        text=feature,
                        xanchor="right",
                        ay=-100,
                        ax=-100,
                        opacity=0.8,
                        arrowhead=0,
                        arrowwidth=0.5,
                        arrowcolor="white",
                    )]
                ))
py.iplot(fig)

plotly.offline.plot(fig, filename = 'output_plots/3Dportrait_tsne_'+feature+'_'+metric+'_'+organism+'.html', auto_open=False)

#### UMAP

In [ ]:
n_neighbors = 10 # balance between local and global structure in the data
spread = 1.
min_dist = 0.1 # defines how dense points are stacked together 
metric='euclidean'

In [ ]:
%%time 
umap_posG3d = embed_umap_3D(G, DM, n_neighbors, spread, min_dist, metric)

In [ ]:
%%time 
#umap_edges = get_trace_edges(G, umap_posG3d_adj, edge_color) 
umap_nodes = get_trace_nodes(G, umap_posG3d_adj,  l_features, colours, size3d)

#umap_data = [umap_adj_edges, umap_adj_nodes]
umap_data = [umap_nodes]

In [ ]:
print('Perplexity:', prplxty3d)
print('Early Exaggeration:', density3d)
print('Learning rate:', l_rate3d)
print('Iterations:', steps3d)


fig = pgo.Figure()
for i in umap_data:
    fig.add_trace(i)
fig.update_layout(template='plotly_dark', showlegend=False, width=2000, height=2000)
py.iplot(fig)

plotly.offline.plot(fig, filename = 'output_plots/3Dportrait_umap_'+feature+'_'+metric+'_'+organism+'.html', auto_open=False)

#### UMAP PORTRAIT EXPORT Coordinates

In [ ]:
# EXPORT as dataframe

df_3Dportrait_umap = pd.DataFrame(umap_posG3d).T
df_3Dportrait_umap.to_csv(r'output_csv/3DPortrait_umap_'+feature+'_'+organism+'_'+'lrate_'+l_rate3d+'steps_'+steps3d+'_'+metric+'.csv', index = True)

In [ ]:
# IMPORT and convert df to dict 
df_umap_posG3d = pd.read_csv('output_csv/3DPortrait_umap_'+feature+'_'+organism+'_'+'lrate_'+l_rate3d+'steps_'+steps3d+'_'+metric+'.csv', index_col=0)
X_umap = list(df_umap_posG3d['0'])
Y_umap = list(df_umap_posG3d['1'])
Z_umap = list(df_umap_posG3d['2'])

umap_posG3d = dict(zip(list(G.nodes()),zip(X_umap,Y_umap,Z_umap)))

## SPHERE

#### UMAP

In [ ]:
metric = 'cosine'

# Radius parameter
d_param = d_essentiality_scores_sorted

In [ ]:
%%time 
umap_sphere = embed_umap_sphere(G, DM, metric)
umap_sphere_withrad = get_posG_with_sphere_radius(G, umap_sphere, d_param)

In [ ]:
%%time
umap_sphere_trace_edges = get_trace_edges(G, umap_sphere_withrad, edge_color)
umap_sphere_trace_nodes = get_trace_umap_sphere(umap_sphere_withrad, l_features, colours, size3d)
umap_sphere_data = [umap_sphere_trace_edges, umap_sphere_trace_nodes]

In [ ]:
print('Metric:', metric)

fig = pgo.Figure()
for i in umap_sphere_data:
    fig.add_trace(i)
fig.update_layout(template='plotly_dark', showlegend=False, width=2000, height=2000)
py.iplot(fig)

plotly.offline.plot(fig, filename = 'output_plots/3Dsphere_umap_'+feature+'_'+metric+'_'+organism+'.html', auto_open=False)

# -----some additional wip stuff-----

### COLOURING SUBCATEGORIES e.g. Disease subcategory (i.e. functional property)

In [ ]:
# input = dict

n = len(sub_categories)
colors = generate_colorlist_nodes(n)

doid_coloured = {}
c=0
for k,v in sub_categories.items():
    doid_coloured[k[1]]=colors[c]
    c+=1
    
d_gene_colours = {}
for doid, gene in d_do_genes.items(): 
    for i in gene:
        for do, col in doid_coloured.items():
            if doid == do:
                d_gene_colours[i]=col

# SORT dict based on G.nodes
d_gene_colours_sorted = {key:d_gene_colours[key] for key in G.nodes()}

l_col_subcat = list(d_gene_colours_sorted.values())


# NODE COLOURING based on Subcategory
colours = l_col_subcat 
edge_color = 'lightgrey'

#sns.palplot(colours)

### COLOURING DISEASE SCORES

In [ ]:
# TO DO : MAKE FUNCTION FOR COLORING 

gene_list = gene_list_retisarc



# COLOURS GENERATED / disease score and neighboring nodes

n = len(gene_list)
color_major = generate_colorlist_nodes(n)
 
factor = 1.7 # the higher the lighter
color_minor = []
for i in color_major:
    r,g,b = hex_to_rgb(i)
    color_light = adjust_color_lightness(r,g,b,factor)
    color_minor.append(color_light)
    
    
# ------------------------------------------------------
# NODES coloured 

# Major nodes coloured
d_col_major = {}
for n in d_major_score.keys():
    for i in color_major:
        d_col_major[n] = i

d_col_minor = {}
for node,col in d_col_major.items():
    for node in d_col_major.keys():
        for nd,neigh in major_neigh.items():
            for n in neigh:
                for i in color_minor:
                    if node==nd and n not in d_col_major.keys():
                        d_col_minor[n] = i
                    
d_col = {**d_col_major,**d_col_minor}

d_grey = {}
for i in G.nodes():
    if i not in d_col.keys():
        d_grey[i] = 'lightgrey'
        
        
d_col_all = {**d_col_major, **d_col_minor, **d_grey}
d_col_all_sorted = {key:d_col_all[key] for key in G.nodes()}

l_col_all = list(d_col_all_sorted.values())

colours = l_col_all

# ------------------------------------------------------
# EDGES coloured 

edge_lst = []
for edge in G.edges():
    for e in edge:
        for node in d_col_major.keys():
            if e == node:
                edge_lst.append(edge)
                
d_col_edges = {}
for e in edge_lst:
    for node,col in d_col_major.items():
        if e[0] == node:
            d_col_edges[e]=col
        elif e[1] == node:
            d_col_edges[e]=col

d_grey_edges = {}
for edge in G.edges():
    if edge not in d_col_edges.keys(): 
        d_grey_edges[edge] = 'lightgrey'
        
d_edges_all = {**d_col_edges, **d_grey_edges}

# Sort according to G.edges()
d_edges_all_sorted = {key:d_edges_all[key] for key in G.edges()}

edge_color = list(d_edges_all_sorted.values())

In [ ]:
'''# CHOOSE DISEASE CATEGORY 
disease_category = 'cancer'

l_disease_genes = []
for d_name in d_names_do.keys():
    if d_name.find(disease_category) != -1:
        #print(d_name,d_names_do[d_name])
        try:
            l_genes = d_do_genes[d_names_do[d_name]]
            for gene in l_genes:
                l_disease_genes.append(gene)
        except:
            #print(d_names_do[d_name],d_name)
                pass
set_disease_genes = set(l_disease_genes)
print('\nThere are %s genes found to be associated with "%s".' %(len(set_disease_genes),disease_category))

sub_categories = {}
for d_name in d_names_do.keys():
    if d_name.find(disease_category) != -1:
        try:
            sub_categories[d_name,d_names_do[d_name]]=len(d_do_genes[d_names_do[d_name]])
            #print('specific disease: %s (%s) ; # associated genes: %s' %(d_name,d_names_do[d_name],len(d_do_genes[d_names_do[d_name]])))
        except:
            pass'''